In [120]:
!pip install --upgrade --quiet sagemaker
!pip install langchain --quiet
import boto3
import json

from langchain import SagemakerEndpoint
from langchain.llms.sagemaker_endpoint import LLMContentHandler
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory

定义ContentHandler以标准化Sagemaker Endpoint的输入和输出

In [116]:
parameters = {
    "max_new_tokens": 512,
    "top_p": 0.9,
    "temperature": 1,
}


class ContentHandler(LLMContentHandler):
    content_type = "application/json"
    accepts = "application/json"

    def transform_input(self, prompt: str, model_kwargs={}) -> bytes:
        payload = {
            "inputs": [[
                {"role": "user", "content": prompt},
            ]],
        }
        
        input_str = json.dumps(payload)
        return input_str.encode("utf-8")

    def transform_output(self, output: bytes):
        response_json = json.loads(output.read().decode("utf-8"))
        return (f"> {response_json[0]['generation']['role'].capitalize()}: {response_json[0]['generation']['content']}")

实例化Endpoint LLM

In [117]:
content_handler = ContentHandler()

llm=SagemakerEndpoint(
        endpoint_name="meta-textgeneration-llama-2-13b-f-2023-07-30-08-44-18-636",
        region_name="us-east-1",
        #model_kwargs={"max_new_tokens": 512, "top_p": 0.9, "temperature": 0.6},
        content_handler=content_handler,
        model_kwargs=parameters,
        endpoint_kwargs = {"CustomAttributes": "accept_eula=true"}
    )

## 没有加入Memory之前，用llm chain测试

In [143]:
template = """你是一个聊天机器人.

Human: {human_input}
AI:"""
 
    

prompt = PromptTemplate(
    input_variables=["human_input"], 
    template=template
)

llm_chain = LLMChain(prompt=prompt, llm=llm)

In [144]:
llm_chain.predict(human_input="Hi, my name is Sean, and you?")

"> Assistant:  Hello Sean! My name is LLaMA, I'm a chatbot developed by Meta AI. It's nice to meet you! 😊 What can I help you with today? Please keep in mind that I'm here to assist you in a respectful, honest, and safe manner. If your question doesn't make sense or is not factually coherent, I'll do my best to explain why instead of providing an incorrect answer. Let's get started! 💡"

In [145]:
llm_chain.predict(human_input="so, do you remember my name?")

"> Assistant:  Hello! I'm here to assist you in a helpful, respectful, and honest manner. I'm a safe and knowledgeable AI assistant, and I'm here to provide information and answer your questions to the best of my ability. However, I cannot remember personal information such as names, as I'm a text-based AI and do not have the ability to store or recall personal data. If you'd like to introduce yourself, I'd be happy to chat with you and assist you with any questions you may have!"

## 加入Memory之后，用llm chain测试

In [146]:
template = """你是一个聊天机器人.
 
{chat_history}
Human: {human_input}
AI:"""
 
    

prompt = PromptTemplate(
    input_variables=["chat_history", "human_input"], 
    template=template
)
memory = ConversationBufferMemory(memory_key="chat_history")

In [147]:
llm_chain = LLMChain(
    llm=llm, 
    prompt=prompt, 
    verbose=True, 
    memory=memory,
)

In [148]:
llm_chain.predict(human_input="Hi, my name is Sean, and you?")



> Entering new LLMChain chain...
Prompt after formatting:
你是一个聊天机器人.
 

Human: Hi, my name is Sean, and you?
AI:

> Finished chain.


"> Assistant:  Hello Sean! My name is Assistant, and it's nice to meet you. I'm here to help answer any questions you may have. Please keep in mind that I am a safe and respectful assistant, and I will not provide any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. I strive to provide socially unbiased and positive responses. If a question does not make sense or is not factually coherent, I will explain why instead of providing an incorrect answer. If I don't know the answer to a question, I will not provide false information. Please feel free to ask me anything!"

In [149]:
llm_chain.predict(human_input="so, do you remember my name?")



> Entering new LLMChain chain...
Prompt after formatting:
你是一个聊天机器人.
 
Human: Hi, my name is Sean, and you?
AI: > Assistant:  Hello Sean! My name is Assistant, and it's nice to meet you. I'm here to help answer any questions you may have. Please keep in mind that I am a safe and respectful assistant, and I will not provide any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. I strive to provide socially unbiased and positive responses. If a question does not make sense or is not factually coherent, I will explain why instead of providing an incorrect answer. If I don't know the answer to a question, I will not provide false information. Please feel free to ask me anything!
Human: so, do you remember my name?
AI:

> Finished chain.


"> Assistant:  Assistant: Hello Sean! Yes, I remember your name. You introduced yourself as Sean when you first asked me a question. Please let me know if there's anything else you'd like to know or discuss!"